# All the parameters that need to be changed

In [1]:
# North Carolina
state_ab = "nc"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [38]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/NC_SMmap2_Statewide.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2023/SL 2023-146 Senate - Shapefile/SL 2023-146.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/SL 2022-4.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [18]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|████████████████████████████████████████| 2662/2662 [00:07<00:00, 340.23it/s]


There are 2 overlaps.
There are 3 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 2677/2677 [00:01<00:00, 2128.32it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


100%|████████████████████████████████████████| 2662/2662 [00:06<00:00, 392.73it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [19]:
list(vest20.columns)

['PREC_ID',
 'ENR_DESC',
 'COUNTY_NAM',
 'COUNTY_ID',
 'G20PRERTRU',
 'G20PREDBID',
 'G20PRELJOR',
 'G20PREGHAW',
 'G20PRECBLA',
 'G20PREOWRI',
 'G20USSRTIL',
 'G20USSDCUN',
 'G20USSLBRA',
 'G20USSCHAY',
 'G20GOVRFOR',
 'G20GOVDCOO',
 'G20GOVLDIF',
 'G20GOVCPIS',
 'G20LTGRROB',
 'G20LTGDHOL',
 'G20ATGRONE',
 'G20ATGDSTE',
 'G20TRERFOL',
 'G20TREDCHA',
 'G20SOSRSYK',
 'G20SOSDMAR',
 'G20AUDRSTR',
 'G20AUDDWOO',
 'G20AGRRTRO',
 'G20AGRDWAD',
 'G20INSRCAU',
 'G20INSDGOO',
 'G20LABRDOB',
 'G20LABDHOL',
 'G20SPIRTRU',
 'G20SPIDMAN',
 'G20SSCRNEW',
 'G20SSCDBEA',
 'G20SSCRBER',
 'G20SSCDINM',
 'G20SSCRBAR',
 'G20SSCDDAV',
 'G20SACRWOO',
 'G20SACDSHI',
 'G20SACRGOR',
 'G20SACDCUB',
 'G20SACRDIL',
 'G20SACDSTY',
 'G20SACRCAR',
 'G20SACDYOU',
 'G20SACRGRI',
 'G20SACDBRO',
 'geometry']

## Parameters that need to be checked

In [20]:
start_col = 4
vest_base_data = vest20
year = '20'

In [21]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [22]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|█████████████████████████████████████████| 2662/2662 [01:15<00:00, 35.44it/s]


In [23]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [24]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [25]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [28]:
check_population(population_df, vest_base)

population agrees


# Add more vest data

In [29]:
vest18 = gpd.read_file(vest18_data)

In [30]:
list(vest18.columns)

['PREC_ID',
 'ENR_DESC',
 'COUNTY_NAM',
 'COUNTY_ID',
 'G18SSCRJAC',
 'G18SSCRANG',
 'G18SSCDEAR',
 'G18SACRHEA',
 'G18SACDARR',
 'G18SACRGRI',
 'G18SACRRAY',
 'G18SACDHAM',
 'G18SACRKIT',
 'G18SACDCOL',
 'G18SACLMON',
 'geometry']

In [34]:
# check the result here
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|████████████████████████████████████████| 2706/2706 [00:06<00:00, 432.46it/s]


There are 4 overlaps.
There are 3 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 2737/2737 [00:01<00:00, 2129.81it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


100%|█████████████████████████████████████████| 2662/2662 [01:04<00:00, 40.97it/s]


population agrees


In [35]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['PREC_ID', 'ENR_DESC', 'COUNTY_NAM', 'COUNTY_ID', 'G16PRERTRU',
       'G16PREDCLI', 'G16PRELJOH', 'G16PREOWRI', 'G16USSRBUR', 'G16USSDROS',
       'G16USSLHAU', 'G16GOVRMCC', 'G16GOVDCOO', 'G16GOVLCEC', 'G16LTGRFOR',
       'G16LTGDCOL', 'G16LTGLCOL', 'G16ATGRNEW', 'G16ATGDSTE', 'G16TRERFOL',
       'G16TREDBLU', 'G16SOSRLAP', 'G16SOSDMAR', 'G16AUDRSTU', 'G16AUDDWOO',
       'G16AGRRTRO', 'G16AGRDSMI', 'G16INSRCAU', 'G16INSDGOO', 'G16LABRBER',
       'G16LABDMEE', 'G16LABOWRI', 'G16SPIRJOH', 'G16SPIDATK', 'G16SSCREDM',
       'G16SSCDMOR', 'G16SACRDIE', 'G16SACDROZ', 'G16SACRMUR', 'G16SACDEAG',
       'G16SACUBUI', 'G16SACRHUN', 'G16SACDJON', 'G16SACRBER', 'G16SACDSTE',
       'G16SACRZAC', 'G16SACDMCK', 'geometry'],
      dtype='object')

In [36]:
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|████████████████████████████████████████| 2704/2704 [00:06<00:00, 426.89it/s]


There are 1 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 2729/2729 [00:01<00:00, 2172.30it/s]


Resolving overlaps...
Filling gaps...


100%|█████████████████████████████████████████| 2662/2662 [01:04<00:00, 41.38it/s]


population agrees


## Add the district data

In [39]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [40]:
cong_df.head()

,OBJECTID,District_A,Shape_Leng,Shape_Area,geometry
0,1,1,8.604113,2.192938,"POLYGON ((-76.91590 36.55215, -76.91555 36.552..."
1,2,2,2.291793,0.131161,"POLYGON ((-78.70273 36.07617, -78.70164 36.076..."
2,3,3,13.787605,2.928661,"POLYGON ((-75.79750 36.55092, -75.79051 36.500..."
3,4,4,3.876041,0.541862,"POLYGON ((-78.45625 36.54109, -78.45588 36.541..."
4,5,5,8.304280,1.184265,"POLYGON ((-81.61951 36.58729, -81.61856 36.587..."


In [42]:
set(cong_df['District_A'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [43]:
election_df = add_district(cong_df, "CD", election_df, "District_A")

100%|█████████████████████████████████████████████| 14/14 [00:07<00:00,  1.77it/s]


In [44]:
send.head()

,DISTRICT,PL20AA_TOT,geometry
0,1,199623,"POLYGON ((-77.89977 36.54460, -77.89942 36.544..."
1,10,215999,"POLYGON ((-78.25597 35.81812, -78.25603 35.817..."
2,11,206121,"POLYGON ((-78.32399 36.54382, -78.32111 36.529..."
3,12,200794,"POLYGON ((-79.09581 35.19207, -79.09630 35.192..."
4,13,198371,"POLYGON ((-78.91473 35.58368, -78.91600 35.584..."


In [45]:
len(set(send['DISTRICT'])) # 50

50

In [46]:
hdist.head()

,DISTRICT,PL20AA_TOT,geometry
0,4,83138,"POLYGON ((-78.30658 35.28760, -78.30642 35.287..."
1,14,90506,"POLYGON ((-77.11720 34.58437, -77.11808 34.584..."
2,15,89176,"POLYGON ((-77.20324 34.83689, -77.20364 34.836..."
3,16,85097,"POLYGON ((-78.16235 34.35701, -78.16219 34.357..."
4,61,90814,"POLYGON ((-79.89761 36.07669, -79.89751 36.076..."


In [48]:
len(set(hdist['DISTRICT'])) # 120

120

In [49]:
election_df = add_district(send, "SEND", election_df, "DISTRICT")

100%|█████████████████████████████████████████████| 50/50 [00:16<00:00,  2.98it/s]


In [50]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|███████████████████████████████████████████| 120/120 [00:15<00:00,  7.85it/s]


In [51]:
list(election_df.columns)

['2MOREVAP',
 'AGR16D',
 'AGR16R',
 'AGR20D',
 'AGR20R',
 'AMINVAP',
 'ASIANVAP',
 'ATG16D',
 'ATG16R',
 'ATG20D',
 'ATG20R',
 'AUD16D',
 'AUD16R',
 'AUD20D',
 'AUD20R',
 'BVAP',
 'COUNTY_ID',
 'COUNTY_NAM',
 'ENR_DESC',
 'GOV16D',
 'GOV16O',
 'GOV16R',
 'GOV20D',
 'GOV20O',
 'GOV20R',
 'HISP',
 'HVAP',
 'H_2MORE',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'INS16D',
 'INS16R',
 'INS20D',
 'INS20R',
 'LAB16D',
 'LAB16O',
 'LAB16R',
 'LAB20D',
 'LAB20R',
 'LTG16D',
 'LTG16O',
 'LTG16R',
 'LTG20D',
 'LTG20R',
 'NHPIVAP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'OTHERVAP',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'PREC_ID',
 'SAC16D',
 'SAC16O',
 'SAC16R',
 'SAC18D',
 'SAC18O',
 'SAC18R',
 'SAC20D',
 'SAC20R',
 'SOS16D',
 'SOS16R',
 'SOS20D',
 'SOS20R',
 'SPI16D',
 'SPI16R',
 'SPI20D',
 'SPI20R',
 'SSC16D',
 'SSC16R',
 'SSC18D',
 'SSC18R',
 'SSC20D',
 'SSC20R',
 'TOTPOP',
 'TRE16D',
 'T

### Put the base precinct year after the precinct information column

In [53]:
base_columns = {
    'PREC_ID': 'PREC_ID' + year,
    'COUNTY_ID': 'COUNTY_ID' + year,
    'COUNTY_NAM': 'COUNTY_NAM' + year,
    'ENR_DESC': 'ENR_DESC' + year,
}
election_df.rename(columns=base_columns, inplace = True)

In [54]:
# reorder the columns
fixed_columns = [
    'PREC_ID' + year,
    'COUNTY_ID' + year,
    'COUNTY_NAM' + year,
    'ENR_DESC' + year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [56]:
list(election_df.columns)

['PREC_ID20',
 'COUNTY_ID20',
 'COUNTY_NAM20',
 'ENR_DESC20',
 'CD',
 'SEND',
 'HDIST',
 'TOTPOP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'HISP',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'H_2MORE',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP',
 'AGR16D',
 'AGR16R',
 'AGR20D',
 'AGR20R',
 'ATG16D',
 'ATG16R',
 'ATG20D',
 'ATG20R',
 'AUD16D',
 'AUD16R',
 'AUD20D',
 'AUD20R',
 'GOV16D',
 'GOV16O',
 'GOV16R',
 'GOV20D',
 'GOV20O',
 'GOV20R',
 'INS16D',
 'INS16R',
 'INS20D',
 'INS20R',
 'LAB16D',
 'LAB16O',
 'LAB16R',
 'LAB20D',
 'LAB20R',
 'LTG16D',
 'LTG16O',
 'LTG16R',
 'LTG20D',
 'LTG20R',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'SAC16D',
 'SAC16O',
 'SAC16R',
 'SAC18D',
 'SAC18O',
 'SAC18R',
 'SAC20D',
 'SAC20R',
 'SOS16D',
 'SOS16R',
 'SOS20D',
 'SOS20R',
 'SPI16D',
 'SPI16R',
 'SPI20D',
 'SPI20R',
 'SSC16D',
 'SSC16R',
 'SSC18

In [57]:
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))